In [18]:
import numpy as np
import random

In [19]:
def SA(initial_state, cost_funct, new_state_funct, L_k = 1, c_k = 100, k = 0.95, min_temp = 0.1, num_iterations = 100, max_or_min = 'max'):
    current_state = initial_state
    current_eval = cost_funct(current_state)
    
    if max_or_min not in ['max', 'min']:
        raise ValueError("Parámetro 'max_or_min' debe ser 'max' o 'min'")
    
    else:
        for _ in range(num_iterations):
            for _ in range(L_k):
                new_state = new_state_funct(current_state)
                new_eval = cost_funct(new_state)
                diff_eval = new_eval - current_eval

                if max_or_min == 'max':
                    if diff_eval >= 0:
                        current_state = new_state
                        current_eval = new_eval
                    else:
                        if np.exp(diff_eval/c_k) > random.random():
                            current_state = new_state
                            current_eval = new_eval

                elif max_or_min == 'min':
                    if diff_eval <= 0:
                        current_state = new_state
                        current_eval = new_eval
                    else:
                        if np.exp(-diff_eval/c_k) > random.random():
                            current_state = new_state
                            current_eval = new_eval

            c_k = max(c_k * k, min_temp)
    return current_state, current_eval

### Knapsack problem

Función heurística:
El objetivo es maximizar el valor total de los artículos seleccionados, sujeto a la restricción de peso.

$$
h(x) = \left\{\begin{matrix}
 -1,\;\;\; \text{if} \;\;\; \sum (w_i \cdot x_i) > \text{Peso máximo}\\
  \sum (v_i \cdot x_i), \;\;\; \text{en caso contrario}
\end{matrix}\right.
$$

In [20]:
n_items = 5 # número de items para elegir
max_weight = 25 # peso máximo permitido
# Define los pesos y valores de los objetos
value_list = np.array([random.randint(1, 50) for _ in range(n_items)])
weight_list = np.array([random.randint(1, 20) for _ in range(n_items)])

In [21]:
#Define el estado inicial aletoriamente
initial_state_KP = np.array([random.randint(0, 1) for _ in range(n_items)])

In [22]:
# Define la función heurística
def h(state, values = value_list, weights = weight_list, maxweight = max_weight):
    value = 0 
    if state @ weights > maxweight:
        value = -1 # Si el valor total de los objetos se pasa del máximo se retorna -1
    else:
        value = state @ values # En caso contrario se retorna el valor total de los objetos
    return value

In [23]:
# Generar vecinos
def neighbors(state):
    newstate = state.copy()
    idx_to_mutate = random.randint(0, n_items - 1)
    newstate[idx_to_mutate] = 1 - newstate[idx_to_mutate]  # Cambia el valor (si 0 -> 1 y si 1 -> 0)
    return newstate

In [24]:
print(f'Estado inicial: {initial_state_KP}')
print(f'Valores de los items: {value_list}')
print(f'Pesos de los items: {weight_list}')
print(f'Peso máximo permitido: {max_weight}')
print(f'Valor del estado incial: {h(initial_state_KP)}')

Estado inicial: [1 0 1 1 0]
Valores de los items: [47 40 19 49  5]
Pesos de los items: [ 1  4  6  8 17]
Peso máximo permitido: 25
Valor del estado incial: 115


In [25]:
KP_result = SA(initial_state_KP, h, neighbors, L_k = 1, c_k = 100, k = 0.9, min_temp = 0.1, num_iterations = 100, max_or_min = 'max')
print(f'Estado final: {KP_result[0]}')
print(f'Valor del estado final: {KP_result[1]}')
print(f'Peso total de los objetos del estado final: {KP_result[0] @ weight_list}')

Estado final: [1 1 1 1 0]
Valor del estado final: 155
Peso total de los objetos del estado final: 19


### Travel Salesman Problem (TSP)

El objetivo es minimizar la distancia total recorrida. De manera que se puede definir la función objetivo como:

$$
f(\pi) = d(\pi_n, \pi_1) + \sum_{i=1}^{n-1} d(\pi_i, \pi_{i+1}),
$$

donde $d(i, j)$ es la distancia entre la ciudad $i$ y la ciudad $j$. Así, el algoritmo acepta la mutación si $f(\pi_{nuevo}) < f(\pi _{actual})$.

In [26]:
n = 5 # Número de ciudades
# Inicialización del estado (tour inicial aleatorio)
initial_state_TSP = random.sample(range(n), n)  # Generamos una permutación aleatoria de ciudades

In [27]:
distance_matrix = np.zeros([n, n])
for i in range(n):
    for j in range(i):
        distance_matrix[i, j] = distance_matrix[j, i] = random.randint(1, 10)

In [28]:
def calculate_total_distance(tour, dist_matrix = distance_matrix):
    total_dist = 0
    for i in range(len(tour)-1):
        total_dist += dist_matrix[tour[i]][tour[i + 1]]
    total_dist += dist_matrix[tour[-1]][tour[0]]
    return total_dist
    
def generate_random_instance(tour):
    new_tour = tour.copy()
    indx1, indx2 = random.sample(range(len(tour)), 2) # Seleciona dos indices aleatoriamente
    new_tour[indx1], new_tour[indx2] = new_tour[indx2], new_tour[indx1] # Se intercambia el valor de los items con los indicies selecionados anteriormente
    return new_tour

In [29]:
print(f'Estado inicial: {initial_state_TSP}')
print(f'Distancia entre caminos: {distance_matrix}')
print(f'Valor del estado inicial: {calculate_total_distance(initial_state_TSP)}')

Estado inicial: [1, 3, 0, 2, 4]
Distancia entre caminos: [[ 0.  9.  3.  1.  2.]
 [ 9.  0.  3. 10.  4.]
 [ 3.  3.  0.  9.  5.]
 [ 1. 10.  9.  0.  6.]
 [ 2.  4.  5.  6.  0.]]
Valor del estado inicial: 23.0


In [30]:
TSP_result = SA(initial_state_TSP, calculate_total_distance, generate_random_instance, L_k = 1, c_k = 100, k = 0.95, min_temp = 0.1, num_iterations = 100, max_or_min = 'min')
print(f'Estado final: {TSP_result[0]}')
print(f'Valor del estado final: {TSP_result[1]}')

Estado final: [1, 4, 3, 0, 2]
Valor del estado final: 17.0


### Función cuadrática (minimizar)

$$f(X) = \sum _{i=1} ^{D} x_i ^2 \text{ ,  con  } -10 <= x_i <= 10$$

In [31]:
D = 5 # Tamño del vector de entrada
initial_state_QF = np.array([round(random.choice([-1, 1]) * random.random() * 10, 3) for _ in range(D)]) # Se genera un vector aleatorio con valores en el rango [-10, 10]

In [32]:
def f(state):
    return sum(state**2) # retorna la suma de los cuadrados de los elementos del estado ingresado
    
def random_new_state(current_state, step_size = 0.1):
    indx = random.randint(0, D - 1)
    new_state = current_state.copy()
    new_state[indx] = random.uniform(-step_size, step_size) #avanza o retrocede un paso dado por step_size de manera aletoria
    return new_state

In [33]:
print(f'Estado inicial: {initial_state_QF}')
print(f'Valor del estado incial: {f(initial_state_QF)}')

Estado inicial: [-3.988 -7.056  8.731 -1.937  4.564]
Valor del estado incial: 166.50370600000002


In [34]:
QF_result = SA(initial_state_QF, f, random_new_state, L_k = D, c_k = 100, k = 0.95, min_temp = 0.1, num_iterations = 100, max_or_min = 'min')
print(f'Estado final: {QF_result[0]}')
print(f'Valor del estado final: {QF_result[1]}')

Estado final: [0.04173621 0.07110204 0.06738724 0.07306602 0.08269206]
Valor del estado final: 0.02351507179434207
